# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

Defaulting to user installation because normal site-packages is not writeable
  Cloning git://github.com/Theano/Theano.git to /tmp/pip-req-build-89lcaaao
  Created wheel for Theano: filename=Theano-1.0.5+1.geb6a4125c-py3-none-any.whl size=2669427 sha256=cd1dbdd0bc607bcc1f248e78394898e8fdee1c09b6dbcdc658d5c5bea17913fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-dqp767uw/wheels/31/0b/3d/ec0a7d000aed975d9ad70a301a3bd2c5fcdab43b1eba72eb59
Successfully built Theano
  Attempting uninstall: Theano
    Found existing installation: Theano 1.0.5+1.geb6a4125c
    Uninstalling Theano-1.0.5+1.geb6a4125c:
      Successfully uninstalled Theano-1.0.5+1.geb6a4125c
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 320.4 MB 4.8 kB/s eta 0:00:019   |████▍                           | 44.4 MB 104 kB/s eta 0:44:01     |█████▏                          | 52.2 MB 1.1 MB/s eta 0:04:04     |█████▎                          | 53.2 MB 1.1 MB/s

  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: pip in /home/maiconmares/.local/lib/python3.6/site-packages (20.2.3)
     |████████████████████████████████| 9.5 MB 671 kB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 302 kB/s eta 0:00:01
     |████████████████████████████████| 11.6 MB 124 kB/s ta 0:00:019    |██▍                             | 849 kB 79 kB/s eta 0:02:15     |████████████████████▊           | 7.5 MB 58 kB/s eta 0:01:11
Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-packages (0.0)
     |████████████████████████████████| 156 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 193 kB/s eta 0:00:01
  Attempting uninstall: certifi
    Found existing installation: certifi 2019.6.16
    Uninstalling certifi-2019

# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), [1])], 
    remainder='passthrough'
)
X = np.array(onehotencoder.fit_transform(X), dtype=np.float)
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [16]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [17]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4873 - accuracy: 0.7958
Epoch 2/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4285 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4231 - accuracy: 0.7960
Epoch 4/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4192 - accuracy: 0.8134
Epoch 5/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4167 - accuracy: 0.8267
Epoch 6/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4145 - accuracy: 0.8275
Epoch 7/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4130 - accuracy: 0.8317
Epoch 8/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4121 - accuracy: 0.8310
Epoch 9/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4112 - accuracy: 0.8324
Epoch 10/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4102 - accura

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [18]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1545   50]
 [ 273  132]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [21]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 2s 2ms/step - loss: 0.5087 - accuracy: 0.7960
Epoch 2/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4811 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4455 - accuracy: 0.7960
Epoch 4/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4424 - accuracy: 0.7960
Epoch 5/100
800/800 [==============================] - 2s 3ms/step - loss: 0.4328 - accuracy: 0.7960
Epoch 6/100
800/800 [==============================] - 2s 3ms/step - loss: 0.4309 - accuracy: 0.7960
Epoch 7/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4306 - accuracy: 0.7960
Epoch 8/100
800/800 [==============================] - 2s 3ms/step - loss: 0.4296 - accuracy: 0.7964
Epoch 9/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4293 - accuracy: 0.7952
Epoch 10/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4287 - accura

800/800 [==============================] - 2s 2ms/step - loss: 0.3654 - accuracy: 0.8461
Epoch 82/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3612 - accuracy: 0.8472
Epoch 83/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3605 - accuracy: 0.8475
Epoch 84/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3607 - accuracy: 0.8478
Epoch 85/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3620 - accuracy: 0.8478: 
Epoch 86/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3616 - accuracy: 0.8479
Epoch 87/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3623 - accuracy: 0.8478
Epoch 88/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3627 - accuracy: 0.8471
Epoch 89/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3598 - accuracy: 0.8482
Epoch 90/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3594 - accuracy

In [22]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [23]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1515   80]
 [ 228  177]]


# Initialising the 3rd ANN

In [24]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 2s 2ms/step - loss: 0.5121 - accuracy: 0.7960
Epoch 2/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4468 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 2s 3ms/step - loss: 0.4343 - accuracy: 0.7960
Epoch 4/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4297 - accuracy: 0.7985
Epoch 5/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4282 - accuracy: 0.8008
Epoch 6/100
800/800 [==============================] - 2s 3ms/step - loss: 0.4276 - accuracy: 0.8023
Epoch 7/100
800/800 [==============================] - 2s 3ms/step - loss: 0.4277 - accuracy: 0.8031
Epoch 8/100
800/800 [==============================] - 2s 3ms/step - loss: 0.4275 - accuracy: 0.8029
Epoch 9/100
800/800 [==============================] - 2s 3ms/step - loss: 0.4274 - accuracy: 0.8036
Epoch 10/100
800/800 [==============================] - 2s 3ms/step - loss: 0.4273 - accura

800/800 [==============================] - 2s 2ms/step - loss: 0.3964 - accuracy: 0.8370
Epoch 82/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3964 - accuracy: 0.8356
Epoch 83/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3962 - accuracy: 0.8360
Epoch 84/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3960 - accuracy: 0.8361
Epoch 85/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3959 - accuracy: 0.8355
Epoch 86/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3956 - accuracy: 0.8364
Epoch 87/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3956 - accuracy: 0.8369
Epoch 88/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3956 - accuracy: 0.8353
Epoch 89/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3955 - accuracy: 0.8359
Epoch 90/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3954 - accuracy: 

In [25]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [26]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1546   49]
 [ 263  142]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [27]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:%s\nmean:%s\nvariance:%s".format(accuracies, mean, variance))

accuracies:%s
mean:%s
variance:%s


# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [28]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Tuning the ANN

In [29]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, PReLU

def build_classifier(optimizer):
    # Adding the input layer and the first hidden layer
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', input_dim = 11))
    classifier.add(PReLU())

    # Adding the second layer
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform'))
    classifier.add(PReLU())


    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [30]:
parameters = {'batch_size': [10, 24, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/50
720/720 [==============================] - 2s 3ms/step - loss: 0.5007 - accuracy: 0.7967
Epoch 2/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4302 - accuracy: 0.8024
Epoch 3/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4222 - accuracy: 0.8175
Epoch 4/50
720/720 [==============================] - 2s 3ms/step - loss: 0.4162 - accuracy: 0.8228
Epoch 5/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4148 - accuracy: 0.8257
Epoch 6/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4132 - accuracy: 0.8301
Epoch 7/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4102 - accuracy: 0.8335
Epoch 8/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4092 - accuracy: 0.8328
Epoch 9/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4069 - accuracy: 0.8332
Epoch 10/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4065 - accuracy: 0.8340

720/720 [==============================] - 2s 2ms/step - loss: 0.4071 - accuracy: 0.8317
Epoch 8/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4059 - accuracy: 0.8319
Epoch 9/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4061 - accuracy: 0.8319
Epoch 10/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4036 - accuracy: 0.8342
Epoch 11/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4032 - accuracy: 0.8347
Epoch 12/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4016 - accuracy: 0.8354
Epoch 13/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4008 - accuracy: 0.8358
Epoch 14/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3993 - accuracy: 0.8343
Epoch 15/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4005 - accuracy: 0.8354
Epoch 16/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3991 - accuracy: 0.8374
Epoc

720/720 [==============================] - 2s 2ms/step - loss: 0.4003 - accuracy: 0.8347
Epoch 39/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4013 - accuracy: 0.8338
Epoch 40/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4001 - accuracy: 0.8354
Epoch 41/50
720/720 [==============================] - 2s 3ms/step - loss: 0.4005 - accuracy: 0.8353
Epoch 42/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4019 - accuracy: 0.8364
Epoch 43/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4008 - accuracy: 0.8350
Epoch 44/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4001 - accuracy: 0.8369
Epoch 45/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3998 - accuracy: 0.8364
Epoch 46/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4005 - accuracy: 0.8351
Epoch 47/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4005 - accuracy: 0.8379
Ep

720/720 [==============================] - 2s 2ms/step - loss: 0.3951 - accuracy: 0.8364
Epoch 1/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4821 - accuracy: 0.7957
Epoch 2/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4285 - accuracy: 0.7957
Epoch 3/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4243 - accuracy: 0.8119
Epoch 4/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4195 - accuracy: 0.8211
Epoch 5/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4149 - accuracy: 0.8271
Epoch 6/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4098 - accuracy: 0.8331
Epoch 7/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4061 - accuracy: 0.8353
Epoch 8/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4052 - accuracy: 0.8344
Epoch 9/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4047 - accuracy: 0.8346
Epoch 10/50

720/720 [==============================] - 2s 2ms/step - loss: 0.4067 - accuracy: 0.8360
Epoch 13/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4044 - accuracy: 0.8354
Epoch 14/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4049 - accuracy: 0.8325
Epoch 15/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4032 - accuracy: 0.8367
Epoch 16/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4020 - accuracy: 0.8347
Epoch 17/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4040 - accuracy: 0.8336
Epoch 18/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4032 - accuracy: 0.8357
Epoch 19/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4014 - accuracy: 0.8350
Epoch 20/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4019 - accuracy: 0.8371
Epoch 21/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4011 - accuracy: 0.8361
Ep

...

720/720 [==============================] - 2s 2ms/step - loss: 0.3400 - accuracy: 0.8626
Epoch 88/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3398 - accuracy: 0.8626
Epoch 89/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3399 - accuracy: 0.8615
Epoch 90/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3379 - accuracy: 0.8649
Epoch 91/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3394 - accuracy: 0.8612
Epoch 92/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3399 - accuracy: 0.8646
Epoch 93/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3392 - accuracy: 0.8647
Epoch 94/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3397 - accuracy: 0.8637
Epoch 95/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3384 - accuracy: 0.8624
Epoch 96/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3389 - accuracy: 

720/720 [==============================] - 2s 2ms/step - loss: 0.3994 - accuracy: 0.8381
Epoch 48/100
720/720 [==============================] - 2s 2ms/step - loss: 0.4001 - accuracy: 0.8371
Epoch 49/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3981 - accuracy: 0.8378
Epoch 50/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3983 - accuracy: 0.8357
Epoch 51/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3987 - accuracy: 0.8354
Epoch 52/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3982 - accuracy: 0.8376
Epoch 53/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3975 - accuracy: 0.8375
Epoch 54/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3994 - accuracy: 0.8360
Epoch 55/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3986 - accuracy: 0.8342
Epoch 56/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3988 - accuracy: 

720/720 [==============================] - 1s 1ms/step - loss: 0.4050 - accuracy: 0.8360
Epoch 8/100
720/720 [==============================] - 2s 2ms/step - loss: 0.4036 - accuracy: 0.8354
Epoch 9/100
720/720 [==============================] - 2s 2ms/step - loss: 0.4030 - accuracy: 0.8363
Epoch 10/100
720/720 [==============================] - 2s 2ms/step - loss: 0.4003 - accuracy: 0.8361
Epoch 11/100
720/720 [==============================] - 12557s 17s/step - loss: 0.4014 - accuracy: 0.8374
Epoch 12/100
720/720 [==============================] - 2s 2ms/step - loss: 0.4006 - accuracy: 0.8374
Epoch 13/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3994 - accuracy: 0.8385
Epoch 14/100
720/720 [==============================] - 2s 3ms/step - loss: 0.3998 - accuracy: 0.8378
Epoch 15/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3986 - accuracy: 0.8368
Epoch 16/100
720/720 [==============================] - 2s 3ms/step - loss: 0.3993 - accuracy

720/720 [==============================] - 2s 2ms/step - loss: 0.3972 - accuracy: 0.8376
Epoch 68/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3969 - accuracy: 0.8392
Epoch 69/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3977 - accuracy: 0.8406
Epoch 70/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3966 - accuracy: 0.8396
Epoch 71/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3966 - accuracy: 0.8378
Epoch 72/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3974 - accuracy: 0.8379
Epoch 73/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3953 - accuracy: 0.8369
Epoch 74/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3956 - accuracy: 0.8399
Epoch 75/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3981 - accuracy: 0.8382
Epoch 76/100
720/720 [==============================] - 2s 2ms/step - loss: 0.3963 - accuracy: 

KeyboardInterrupt: 

# FIM

In [ ]:
> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)